In [1]:
from absbox import mkDeal,API,Generic
#localAPI = API("https://spv.run/api/dev",lang='english',check=False)
localAPI = API("http://localhost:8081",lang='english',check=False)

✅Connected, local lib:0.25.0, server:0.25.0

In [54]:
localAPI = API("https://absbox.org/api/dev")
#localAPI = API("http://localhost:8081",lang='english',check=False)
#library_url = "https://absbox.org/library/latest"

assets = [["FixedAsset"
            ,{"start":"2023-12-01","originBalance":1000000000
              ,"originTerm":38,"residual":1000000000
              ,"period":"Monthly"
              ,"amortize":"Straight"
              ,"capacity":("Fixed",1000000000*0.066/12)}
            ,{"remainTerm":38}
          ]
         ]

exps = (("maintenance",{"type":{"recurFee":["YearFirst",0]}})
        ,("maintenance2",{"type":{"recurFee":["YearFirst",0],"feeStart":"2024-01-03"}})
        ,("insurance",{"type":{"recurFee":["MonthFirst",0]}})
       )

waterfall = {
    "amortizing":[
        ["payFee","acc01",['maintenance','maintenance2','insurance']]
        ,["accrueAndPayInt","acc01",["A"]]
        ,["payPrin","acc01",["A"]]
        ,["accrueAndPayInt","acc01",["EQ"]]
        ,["payPrin","acc01",["EQ"]
            ,{"limit":{"formula":("floorWithZero"
            ,("substract"
            ,("accountBalance","acc01"),("constant",1045)))}}]
    ]
    ,"cleanUp":[
    ["sellAsset",["Current|Defaulted",1.0,0],"acc01"]
    ,["payPrin","acc01",["EQ"]]
    ,["payIntResidual","acc01","EQ"]
    ]
}

SRT = Generic(
"SRT"
,{"cutoff":"2024-01-01","closing":"2024-01-03","firstPay":"2024-02-01"
,"payFreq":"MonthEnd","poolFreq":"MonthEnd","stated":"2028-01-01"}
,{'assets':assets}
    ,(("acc01",{"balance":0}),)
    ,(("A",{"balance":900000000
            ,"rate":0.02
            ,"originBalance":900000000
            ,"originRate":0.00
            ,"startDate":"2024-01-02"
            ,"rateType":{"Fixed":0.05}
            ,"bondType":{"Sequential":None}})
    ,("EQ",{"balance":100000000
            ,"rate":0.1
            ,"originBalance":100000000
            ,"originRate":0.1
            ,"startDate":"2024-01-02"
            ,"rateType":{"Fixed":0.1}
            ,"bondType":{"Sequential":None}})
    )
    ,exps
    ,waterfall
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
)

myAssump = ("Pool"
                ,("Fixed",[["2023-01-01",0.7]
                            ,["2025-01-01",0.6]
                            ,["2026-01-01",0.5]]
                          ,[["2023-01-01",0.56]
                           ,["2025-01-01",0.8]
                           ,["2026-01-01",0.56]]
                 )
                ,None
                ,None)

p = localAPI.run(SRT,poolAssump=myAssump
                ,runAssump=[
                    ("call",{"afterDate":"2028-01-01"})
                    ]
                ,read=True)

from absbox.local.analytics import irr 

irr(p['bonds']['EQ'],init=('2024-02-01',-100000000))

⠸

Warning Message from server:
Bond A is not paid off
Bond EQ is not paid off

1.112190554089161

In [52]:
1000000000+(-100000000)

900000000

In [49]:
p['bonds']['EQ']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-02-01,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-02-29,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-03-31,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-04-30,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-05-31,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-06-30,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-07-31,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-08-31,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"
2024-09-30,100000000,0,0,0.1,0,"[<PayInt:EQ>, <PayPrin:EQ>]"


In [51]:
p['bonds']['A']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-02-01,8.971181e+08,1430136.98,2881863.02,0.02,4312000,"[<PayInt:A>, <PayPrin:A>]"
2024-02-29,8.963385e+08,1376400.42,779599.58,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"
2024-03-31,8.957051e+08,1522547.65,633452.35,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"
2024-04-30,8.950215e+08,1472391.92,683608.08,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"
2024-05-31,8.943858e+08,1520310.45,635689.55,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"
2024-06-30,8.937000e+08,1470223.21,685776.79,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"
2024-07-31,8.930621e+08,1518065.77,637934.23,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"
2024-08-31,8.924231e+08,1516982.15,639017.85,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"
2024-09-30,8.917341e+08,1466996.80,689003.20,0.02,2156000,"[<PayInt:A>, <PayPrin:A>]"


The key questions I'm trying to answer are: 

## How do we price structuring nuances? in particular, that we don't care about underlying yield (and we don't benefit from excess spread)


-----

`nuance` you mean notes bought by SRT investor ? 

a typical method is DCF on notes' cashflow.

once API.run() return a result , you can retrieve bond cashflow via 

`resp['bonds']['bondid']`, then you can have PV on it using your own curve.

.....

if the note doesn't benifit from excess spread, you can just model the bond as normal sequential bond with a fixed coupon rate. ref: https://absbox-doc.readthedocs.io/en/latest/modeling.html#fix-rate

....
if you think there is a interest shortfall on note if there is insufficent cashflow from the pool, then you can model a third-party who will ganrantee note's interest payment, here is the example:https://absbox-doc.readthedocs.io/en/latest/modeling.html#liquidity-provider-insurance-ganrantee

in the waterfall action, liquidity facility has ability to pay the interest shortfall of the notes, https://absbox-doc.readthedocs.io/en/latest/modeling.html#liquidtiy-facility . which ensure the notes is always getting the fixed coupon payment.



## How do we price replenishment rights?


Does you refer to `revolving feature` of the deal ? like the sponsor's right to put assets in the reference pool ?


### Just the "Right"

if you are asking the yes /no feature of revolving, on the very high level , the risk will be WAL of SRT/Note will be much longer.

### The "right" of putting risk assets in the pool

Like you want price differnt level of risk assets ( high PD, low PD) or long term/short term assets were put into the reference pool during the replenishment period.

I've written a post:https://www.linkedin.com/pulse/peek-revolving-structural-features-bwm-auto-abs-deal-xiaoyu-zhang/?trackingId=GtHIr47ZujT%2B12rWajI00A%3D%3D

code maybe outdated,but the idea is the same.

Given different scenario ,you can price the bond see the difference, which is the impact of `replenishment`

### beyong scope

the replenishment may indicate the credit risk of bank/sponsor etc. that's another topic though.

## Does where the collateral is held matter?


I'm assume you are refering  whether collateral is being hold at SPV or sponsor/originator/the bank ?

the answer depends how you model SRT to a note or a liquidity provider

### model collateral in SPV and assume SRT is a mezzine/junior note

That's the easy way but not 100% reflect how SRT works. As the cashflow generate from SPV/collateral will flow into senior tranche ,then mezzine/junior note . the later notes was modeled as fix coupon rate.(probably the interest payment is calculate on pool balance instead of junior note balance ? I'm assume SRT is more like a insurance protection, thus if it is covering whole credit risk of pool, it shall charge a premium on the size of pool balance ?)

### model collateral in SPV and assume SRT is ganrantee

thus the cashflow of SRT is not a bond cashflow but a cashflow of liquidity provider: https://absbox-doc.readthedocs.io/en/latest/modeling.html#liquidity-provider

the liquidity provider has ability to charge a fee/premium base on a formula and it can be modeled to payout cash to SPV/Accounts if there is a default in the reference pool.

Either way, you are analysising cashflow from notes or liquidity provider.

## Jump to zero risk based on tranche size?


Could you pls elaberate on the `zero risk based on tranche size` ? 

## How do specific structural features of the deal impact the range of potential outcomes?

basically, I think it can be achieve via `sensitivity analysis`, you can build deal with different structural features and compare the results.

you can refer to this : https://absbox-doc.readthedocs.io/en/latest/analytics.html#multi-structs
        
just call API.runStructus(), then a map will be returned, you can just compare the cashflow difference in the maps.

there are couple candy functions to help you relief the process of tweaking components of deal object. pls refer to https://absbox-doc.readthedocs.io/en/latest/howto.html#how-to-structuring-a-deal